### Jupyter Notebook: Unit Tests for `apply_strategy_rules`

#### Cell 1: Setup - Function and Imports

First, we need to define the function we are testing and import the necessary libraries.

In [1]:
import pandas as pd
import numpy as np
import operator

def apply_strategy_rules(features, rules, config):
    """
    Applies a list of filtering rules to a features DataFrame.

    Args:
        features (pd.DataFrame): The DataFrame containing all calculated features.
        rules (list): A list of dictionaries, where each dict defines a filtering rule.
        config (dict): The configuration dictionary, used for dynamic thresholds.

    Returns:
        pd.Series: A boolean Series (mask) indicating which rows pass all rules.
    """
    # Start with a mask that is True for all rows. We will progressively filter it.
    final_mask = pd.Series(True, index=features.index)
    
    # Map operator strings to actual Python operator functions for flexibility
    op_map = {
        '>': operator.gt,
        '<': operator.lt,
        '>=': operator.ge,
        '<=': operator.le,
        '==': operator.eq,
        '!=': operator.ne
    }

    for rule in rules:
        op_func = op_map[rule['operator']]
        
        # --- Rule Type 1: Comparing two columns ---
        if 'column_A' in rule and 'column_B' in rule:
            mask = op_func(features[rule['column_A']], features[rule['column_B']])
        
        # --- Rule Type 2: Comparing a column to a value ---
        elif 'column' in rule:
            # Determine the value to compare against
            if 'value' in rule:
                value = rule['value']
            elif 'value_from_config' in rule:
                value = config[rule['value_from_config']]
            else:
                raise ValueError(f"Rule missing 'value' or 'value_from_config': {rule}")
            
            mask = op_func(features[rule['column']], value)
            
        else:
            raise ValueError(f"Invalid rule format: {rule}")
        
        # Combine the mask for this rule with the final mask using a logical AND
        final_mask &= mask
            
    return final_mask

print("Setup complete. The 'apply_strategy_rules' function is defined.")



Setup complete. The 'apply_strategy_rules' function is defined.


#### Cell 2: Test 1.1 - Operator Logic ('>')

We'll start by verifying the most basic comparison: greater than.

In [2]:
print("--- Test 1.1: Verifying Operator Logic ('>') ---")
# Arrange: Create a predictable DataFrame
test_df = pd.DataFrame({'A': [-1, 0, 1, 5, 6]})
rule = [{'column': 'A', 'operator': '>', 'value': 1}]
config = {}

# Act: Apply the rule
mask = apply_strategy_rules(test_df, rule, config)
result_df = test_df[mask]

# Assert: Check if the output is exactly what we expect
expected_indices = [3, 4]
try:
    assert result_df.index.tolist() == expected_indices
    print("[SUCCESS]: Operator '>' works as expected.")
    print(f'\nrule:\n{rule}')    
    print(f'\ntest_df:\n{test_df}')
    print(f'\nexpected_indices:\n{expected_indices}')    
except AssertionError:
    print("[FAILURE]: Operator '>' produced an incorrect result.")
    print("Expected indices:", expected_indices)
    print("Got indices:", result_df.index.tolist())


--- Test 1.1: Verifying Operator Logic ('>') ---
[SUCCESS]: Operator '>' works as expected.

rule:
[{'column': 'A', 'operator': '>', 'value': 1}]

test_df:
   A
0 -1
1  0
2  1
3  5
4  6

expected_indices:
[3, 4]


#### Cell 3: Test 1.2 - All Other Operators

Let's write a small loop to efficiently test all the other operators.

In [3]:
test_df = pd.DataFrame({'A': [4.9, 5.0, 5.1]})

# Define test cases: [rule, expected_indices]
operator_tests = [
    ([{'column': 'A', 'operator': '<', 'value': 5.0}],  [0]),
    ([{'column': 'A', 'operator': '>=', 'value': 5.0}], [1, 2]),
    ([{'column': 'A', 'operator': '<=', 'value': 5.0}], [0, 1]),
    ([{'column': 'A', 'operator': '==', 'value': 5.0}], [1]),
    ([{'column': 'A', 'operator': '!=', 'value': 5.0}], [0, 2]),
]

all_passed = True
for rule, expected_indices in operator_tests:
    mask = apply_strategy_rules(test_df, rule, {})
    result_indices = test_df[mask].index.tolist()
    
    op = rule[0]['operator']
    if result_indices != expected_indices:
        print(f"[FAILURE]: Operator '{op}' produced an incorrect result.")
        print(f"  Expected indices: {expected_indices}, Got: {result_indices}")
        all_passed = False
    else:
        print(f"passed")    
        print(f'\nrule:\n{rule}')    
        print(f'\ntest_df:\n{test_df}')
        print(f'\nexpected_indices:\n{expected_indices}')
        print(f'{"="*10}\n')      


if all_passed:
    print("[SUCCESS]: All operators ('<', '>=', '<=', '==', '!=') work as expected.")
    

passed

rule:
[{'column': 'A', 'operator': '<', 'value': 5.0}]

test_df:
     A
0  4.9
1  5.0
2  5.1

expected_indices:
[0]

passed

rule:
[{'column': 'A', 'operator': '>=', 'value': 5.0}]

test_df:
     A
0  4.9
1  5.0
2  5.1

expected_indices:
[1, 2]

passed

rule:
[{'column': 'A', 'operator': '<=', 'value': 5.0}]

test_df:
     A
0  4.9
1  5.0
2  5.1

expected_indices:
[0, 1]

passed

rule:
[{'column': 'A', 'operator': '==', 'value': 5.0}]

test_df:
     A
0  4.9
1  5.0
2  5.1

expected_indices:
[1]

passed

rule:
[{'column': 'A', 'operator': '!=', 'value': 5.0}]

test_df:
     A
0  4.9
1  5.0
2  5.1

expected_indices:
[0, 2]

[SUCCESS]: All operators ('<', '>=', '<=', '==', '!=') work as expected.


#### Cell 4: Test 2 - Value Sources (Static vs. Config)

This test ensures that getting a value from the `config` dictionary works identically to a hardcoded `value`.

In [4]:
print("\n--- Test 2: Verifying Value Sources ---")
# Arrange
test_df = pd.DataFrame({'A': [9, 10, 11]})
config = {'my_threshold': 10}

# Act
rule_static = [{'column': 'A', 'operator': '>', 'value': 10}]
rule_config = [{'column': 'A', 'operator': '>', 'value_from_config': 'my_threshold'}]

mask_static = apply_strategy_rules(test_df, rule_static, config)
mask_config = apply_strategy_rules(test_df, rule_config, config)

# Assert
try:
    pd.testing.assert_series_equal(mask_static, mask_config)
    print("[SUCCESS]: 'value_from_config' works identically to a static 'value'.")
    print(f"passed")    
    print(f'\nrule:\n{rule}')    
    print(f'\ntest_df:\n{test_df}')
    print(f'\nmask_static:\n{mask_static}') 
    print(f'\nmask_config:\n{mask_config}')          
    print(f'{"="*10}\n') 

except AssertionError:
    print("[FAILURE]: Results from 'value_from_config' and static 'value' do not match.")


--- Test 2: Verifying Value Sources ---
[SUCCESS]: 'value_from_config' works identically to a static 'value'.
passed

rule:
[{'column': 'A', 'operator': '!=', 'value': 5.0}]

test_df:
    A
0   9
1  10
2  11

mask_static:
0    False
1    False
2     True
dtype: bool

mask_config:
0    False
1    False
2     True
dtype: bool



#### Cell 5: Test 3 - Column-vs-Column Logic

Here we verify that comparing two columns works correctly.

In [5]:
print("\n--- Test 3: Verifying Column-vs-Column Logic ---")
# Arrange
test_df = pd.DataFrame({
    'A': [10, 20, 30, 40],
    'B': [15, 15, 35, 35]
})
rule = [{'column_A': 'A', 'operator': '>', 'column_B': 'B'}]

# Act
mask = apply_strategy_rules(test_df, rule, {})
result_df = test_df[mask]

# Assert
expected_indices = [1, 3] # Indices where A > B
try:
    assert result_df.index.tolist() == expected_indices
    print("[SUCCESS]: Column-vs-column comparison works as expected.")
    print(f"passed")    
    print(f'\nrule:\n{rule}')    
    print(f'\ntest_df:\n{test_df}')
    print(f'\nresult_df:\n{result_df}')
    print(f'{"="*10}\n')       
except AssertionError:
    print("[FAILURE]: Column-vs-column comparison produced an incorrect result.")
    print("Expected indices:", expected_indices)
    print("Got indices:", result_df.index.tolist())


--- Test 3: Verifying Column-vs-Column Logic ---
[SUCCESS]: Column-vs-column comparison works as expected.
passed

rule:
[{'column_A': 'A', 'operator': '>', 'column_B': 'B'}]

test_df:
    A   B
0  10  15
1  20  15
2  30  35
3  40  35

result_df:
    A   B
1  20  15
3  40  35



#### Cell 6: Test 4 - Edge Cases (`NaN` and Empty Input)

Finally, we test for robustness against missing data and empty inputs.

In [6]:
print("\n--- Test 4: Verifying Edge Cases ---")

# --- 4.1: NaN Handling ---
test_df_nan = pd.DataFrame({'A': [9, np.nan, 11]})
rule = [{'column': 'A', 'operator': '>', 'value': 10}]

mask_nan = apply_strategy_rules(test_df_nan, rule, {})
result_df_nan = test_df_nan[mask_nan]

try:
    # A NaN value should never satisfy a condition
    assert np.nan not in result_df_nan['A'].values
    # Check that only the valid row was returned
    assert result_df_nan.index.tolist() == [2]
    print("[SUCCESS]: NaN values are correctly filtered out.")
    print(f'\nrule:\n{rule}')      
    print(f'\ntest_df_nan:\n{test_df_nan}')
    print(f'\nresult_df_nan:\n{result_df_nan}')
    print(f'{"="*10}\n')             
except AssertionError:
    print("[FAILURE]: NaN values were not handled correctly.")
    
# --- 4.2: Empty DataFrame Input ---
test_df_empty = pd.DataFrame({'A': []})
rule = [{'column': 'A', 'operator': '>', 'value': 10}]

try:
    mask_empty = apply_strategy_rules(test_df_empty, rule, {})
    assert mask_empty.empty
    print("[SUCCESS]: Empty DataFrame input is handled gracefully.")
    print(f'\nrule:\n{rule}')      
    print(f'\ntest_df_empty:\n{test_df_empty}')
    print(f'\nmask_empty:\n{mask_empty}')        
except Exception as e:
    print(f"[FAILURE]: Empty DataFrame input caused an error: {e}")


--- Test 4: Verifying Edge Cases ---
[SUCCESS]: NaN values are correctly filtered out.

rule:
[{'column': 'A', 'operator': '>', 'value': 10}]

test_df_nan:
      A
0   9.0
1   NaN
2  11.0

result_df_nan:
      A
2  11.0

[SUCCESS]: Empty DataFrame input is handled gracefully.

rule:
[{'column': 'A', 'operator': '>', 'value': 10}]

test_df_empty:
Empty DataFrame
Columns: [A]
Index: []

mask_empty:
Series([], dtype: bool)


### By running these cells, you can systematically verify each piece of your rule engine's logic. If all tests pass, you can be highly confident that it is robust and correct.